In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import pprint
import os
import clip
import imagehash
import cv2
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import pickle
import shutil
import torch


In [2]:
train_data_dirs = []

# file path to where your train.json is stored in your machine
train_data_map = "../../train.json"

# file path to where your val.json is stored in your machine
val_data_map = "../../val.json"

# file path to the folder where your train images are stored in your machine
image_folder_dir = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge"

# file path to the folder where your validaion images are stored in your machine
val_data_dir = "../../val_imgs"


In [3]:
with open(train_data_map, 'r') as f:
    train_data = json.load(f)
with open(val_data_map, 'r') as f:
    val_data = json.load(f)

In [4]:
train_df = pd.DataFrame(train_data)
print("shape of data",train_df.shape)
print("count of missiing values\n",train_df.isna().sum())
train_df.head()

shape of data (4099419, 2)
count of missiing values
 caption    0
product    0
dtype: int64


,caption,product
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa


In [5]:
all_files_df = []
sum_all_img = 0
for i in range(9):
    file_df = pd.DataFrame()
    folder = f"{image_folder_dir}/train_text_img_pairs_{i}/train_text_img_pairs_{i}_compressed/"
    file_df["product"] = os.listdir(folder)
    file_df["image_file_path"] = folder
    file_df["image_parent_folder"] = f"train_text_img_pairs_{i}/train_text_img_pairs_{i}_compressed/"
    file_df["image_reletive_folder"] = f"train_text_img_pairs_{i}_compressed/"


    print(file_df.shape)
    sum_all_img += file_df.shape[0]
    all_files_df.append(file_df)

all_files_df= pd.concat(all_files_df)

(486971, 4)
(483293, 4)
(480947, 4)
(413462, 4)
(415712, 4)
(485469, 4)
(485522, 4)
(424338, 4)
(423219, 4)


In [6]:
all_files_df.head()


(4098933, 4)


In [7]:
image_df = pd.merge(train_df,all_files_df,on="product",how="inner")

In [8]:
image_df.shape

(4100105, 5)

In [9]:
image_df["caption_word_count"] = image_df["caption"].str.split(" ").str.len()

In [10]:
image_df["caption_word_count"].max()

68

In [11]:
image_df["caption_word_count"].min()

1

In [12]:
image_df["caption_word_count"].mean()

18.40867075355387

In [13]:
image_df.isna().sum()

caption                  0
product                  0
image_file_path          0
image_parent_folder      0
image_reletive_folder    0
caption_word_count       0
dtype: int64

In [14]:
image_df.head()

,caption,product,image_file_path,image_parent_folder,image_reletive_folder,caption_word_count
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,20
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,14
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,16
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,19
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,30


In [15]:
image_df["img_ext"] = image_df["product"].str.slice(-4)
image_df["img_ext"] = image_df["img_ext"].str.strip(".")

In [16]:
image_df[image_df["img_ext"]=="jpeg"]

,caption,product,image_file_path,image_parent_folder,image_reletive_folder,caption_word_count,img_ext
1938,.. Baby Leggings Spring and Autumn Cotton Thin...,O1CN01tnRIcl1JAXwDRWnbo_!!4197080988-0-tblite....,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,20,jpeg
1977,.. Pajamas Female Autumn and Winter New Plush ...,O1CN016PmuuN1TEnmoBGdRX_!!2206410852351-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,17,jpeg
4212,. Plus Velvet Shirt nv long johns Outer Wear I...,O1CN01ZdRplr241eC1I8rGL_!!2206648367331-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,20,jpeg
4662,. Card Tour Altman Card Blind Box Blind Packag...,O1CN01lpWPj01wu8OZRpFt8_!!2209803276367-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,22,jpeg
9391,. xin kuan man pu fang jian papa dian er tong ...,O1CN01BJ03NN1TLDJr3cxrM_!!2209208362365-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,27,jpeg
...,...,...,...,...,...,...,...
4081160,Secret Braised Pork in Brown Sauce Ginseng Pig...,O1CN0142JKmB2HGqOQtSkxx_!!2211910599124-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,29,jpeg
4083935,The Air Conditioner Washed Cotton summer quilt...,O1CN01B71N8Q1c7Ju7z83WK_!!2207969863553-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,19,jpeg
4084226,Wear String Artifact Once Wear 49 String Wear ...,O1CN01hANXe22Dhd6yeti38_!!2209450868641-0-tbli...,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,19,jpeg
4085311,Curtain Rod Curtain Track Mute Straight Rail W...,O1CN01QpeZcD1EmwzPFnIQ5_!!2612160395.jpeg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,19,jpeg


In [17]:
image_df[image_df["img_ext"]=="gif"]

,caption,product,image_file_path,image_parent_folder,image_reletive_folder,caption_word_count,img_ext
32419,19 The New Harvard M6 Front and Rear Bumpers O...,O1CN01BbfZA81wde3O2SdIO_!!3242356331.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,21,gif
38355,Encryption Large Circular Folding Meals Cover ...,TB2tDgLa0.LL1JjSZFEXXcVmXXa_!!158309702.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,16,gif
41669,Animation Movement Rule Anime Characters GIF D...,O1CN01WMo47M21J6UgS1Drh_!!2206862136963.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,17,gif
52516,21 Autumn and Winter New V-neck Cashmere out C...,O1CN01mjVjil20CP6NLqpox_!!3888736813.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,19,gif
61593,36 * 40mm Power Outlet Module Double Control S...,O1CN01FKDwL827vTHQkPdbH_!!226607859.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,train_text_img_pairs_0_compressed/,16,gif
...,...,...,...,...,...,...,...
4076435,Autumn and Winter New Lapel Sweater Long-Sleev...,O1CN01gv5xjh20CP2jYH4q9_!!3888736813.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,18,gif
4076789,qiu dong kuan yang rong shan zhang xiu ban gao...,O1CN01Ke4CM920COyK8U9fe_!!3888736813.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,30,gif
4077207,Autumn and Winter Cashmere Sweater Half Turtle...,O1CN01hmth8D1LbEJTwnuQj_!!3848281317.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,18,gif
4090249,2021 the New Tea chu xia Charm Tea Yichang Tea...,TB2LyeYbCqJ.eBjy1zbXXbx8FXa_!!2071024595.gif,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_8/train_text_img_pairs_8_...,train_text_img_pairs_8_compressed/,17,gif


In [18]:
image_df_no_gif = image_df.drop(image_df[image_df["img_ext"]=="gif"].index)

In [ ]:
image_df_no_gif["full_path"] = image_df_no_gif["image_file_path"]+ image_df_no_gif["product"]

In [ ]:
image_df_no_gif.to_csv("train_data_v1.csv",index=False)

In [48]:

# from concurrent.futures import ThreadPoolExecutor
# hashes = []
# for path in tqdm(all_images_path, total=len(all_images_path)):
#     image = cv2.imread(path)
#     image = Image.fromarray(image)
#     hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))
# filename = "image_hashes.pkl"
# file = open(filename,"wb")
# pickle.dump(hashes,file)
# file.close()


# def hash_img(path):
#     funcs = [
#     imagehash.average_hash,
#     imagehash.phash,
#     imagehash.dhash,
#     imagehash.whash,
#     #lambda x: imagehash.whash(x, mode='db4'),
#     ]
#     image = cv2.imread(path)
#     image = Image.fromarray(image)
#     hash = np.array([f(image).hash for f in funcs]).reshape(256)
#     return hash
#all_images_path = image_df_no_gif["full_path"].values
# with ThreadPoolExecutor(max_workers=32) as executor:
#     hashes = executor.map(hash_img,all_images_path)
# filename = "image_hashes.pkl"
# file = open(filename,"wb")
# pickle.dump(all_hashes,file)
# all_hashes = list(hashes)
# file.close()


In [49]:
val_df = pd.DataFrame(val_data)
print("shape of data",val_df.shape)
print("count of missiing values\n",val_df.isna().sum())
val_df.to_csv("../data/val_data_map.csv")
val_df.head()

shape of data (50000, 2)
count of missiing values
 caption    0
product    0
dtype: int64


,caption,product
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg


In [50]:
val_df["caption_word_count"] =val_df["caption"].str.len()
val_df["caption_word_count"].max()

391

In [51]:
print("number of unique labels in train data",len(train_df["caption"].unique()))

number of unique labels in train data 3902657


In [52]:
print("number of unique labels in validiation data",len(val_df["caption"].unique()))

number of unique labels in validiation data 49715


In [53]:
train_df["caption"].value_counts()

Ghost mie zhi Edge Perimeter mercari Purchasing Direct Mail                                                                     28
Magic Fairy War Surrounding Stickers mercari Purchasing Direct Mail                                                             23
Off-the-Shelf Japan Imported Second-Hand Inflatable Lighter Have Mars Non-Arcing to Repair Sell Nothing in Return               19
Solid Wood Children with Guardrail Little Baby Boys and Girls Princess Bed Single Side Bed Widening Splicing Queen-Size Beds    19
Hook Strong zhan jiao tie Wall Wall-Mounted Load-Bearing Chuck Kitchen Hook Incognito Paste Door Free Punch Hook                17
                                                                                                                                ..
Tea Pot Cup Glass Filter Set Household Canister Purple Tank Tea Boiled Tea Leaf Gansu Tianshui Electric Furnace.                 1
Tea Retro Blue Glaze Hand-Painted ju ban Master Cup Kung Fu Tea Mug Tea ya shou bei

In [3]:

# filename = "image_hashes.pkl"
# file = open(filename,"rb")
# hashes = pickle.load(file)
# file.close()

In [5]:
# use cuda to speed up
# hashes = torch.Tensor(np.array(hashes).astype(int)).cuda()

# calc similarity scores


In [6]:
# hashes

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')

In [9]:
# sims = np.array([(hashes[i] == hashes).sum(dim=1).detach().cpu().numpy()/256 for i in range(1)])

RuntimeError: CUDA out of memory. Tried to allocate 7.82 GiB (GPU 0; 11.91 GiB total capacity; 4.92 GiB already allocated; 6.20 GiB free; 4.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF